## Python Processing for CHIR Data ##

In [ ]:
# set-up steps
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import warnings
import time
import os

warnings.filterwarnings("ignore")